In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit                        # Funktionsfit:     popt, pcov = curve_fit(func, xdata, ydata) 
from uncertainties import ufloat                            # Fehler:           fehlerwert =  ulfaot(x, err)
from uncertainties.unumpy import uarray                     # Array von Fehler: fehlerarray =  uarray(array, errarray)
from uncertainties.unumpy import (nominal_values as noms,   # Wert:             noms(fehlerwert) = x
                                  std_devs as stds)         # Abweichung:       stds(fehlerarray) = errarray
from uncertainties import unumpy as unp 
import scipy.constants as const

Tabelle erstellen:

In [2]:
md = pd.read_csv('tables/mess1.csv')
md_a = md.iloc[:, [0,1]]
print(md.to_latex(index = False, column_format= "c c c c", decimal=',')) 

\begin{tabular}{c c c c}
\toprule
 alpha &     I &  alpha.1 &   I.1 \\
\midrule
   6,0 & 0,013 &       46 & 0,083 \\
   8,0 & 0,015 &       48 & 0,086 \\
  10,0 & 0,015 &       50 & 0,088 \\
  12,0 & 0,020 &       52 & 0,093 \\
  14,0 & 0,021 &       54 & 0,096 \\
  16,0 & 0,025 &       56 & 0,100 \\
  18,0 & 0,027 &       58 & 0,106 \\
  20,0 & 0,030 &       60 & 0,106 \\
  22,0 & 0,033 &       62 & 0,113 \\
  24,0 & 0,038 &       64 & 0,116 \\
  26,0 & 0,040 &       66 & 0,120 \\
  28,0 & 0,047 &       68 & 0,126 \\
  30,0 & 0,050 &       70 & 0,130 \\
  32,0 & 0,058 &       72 & 0,130 \\
  34,0 & 0,060 &       74 & 0,140 \\
  36,0 & 0,066 &       76 & 0,142 \\
  38,0 & 0,068 &       78 & 0,146 \\
  40,0 & 0,070 &       80 & 0,160 \\
  42,0 & 0,080 &       82 & 0,166 \\
  44,0 & 0,080 &       84 & 0,166 \\
   NaN &   NaN &       86 & 0,166 \\
   NaN &   NaN &       88 & 0,166 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1950/2217053273.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(md.to_latex(index = False, column_format= "c c c c", decimal=','))


Daten auswerten:

Berechnung des Brechungsindex

In [25]:
def func(I):            # Berechnung von (gemessener Strom) / (Referenzstrom)
    return (I / 0.167)

md1 = pd.DataFrame(md).to_numpy()

werte1 = func(md1[:-2,1])
werte2 = func(md1[:,3])


# md1 = werte2
# md1 = pd.DataFrame(md1, columns = ['Column_A'])
# print(md1.to_latex(index = False, column_format= "c", decimal=',')) 

In [53]:
def func2(E, alpha):
    return (np.sqrt(1 + (4 * E * (np.cos(alpha))**2) / (E - 1)**2))

md1 = pd.DataFrame(md).to_numpy()

n1 = func2(werte1, md1[:-2,0] * np.pi / 180)
n2 = func2(werte2, md1[:,2] * np.pi / 180)

nmittel = np.mean(np.append(n1,n2))
nstd = np.std(np.append(n1,n2))
print('Der Mittelwert für n beträgt: ' +  str(nmittel) + ' ± ' + str(nstd))

prin = n2
prin = pd.DataFrame(prin, columns = ['Column_A'])
print(prin.to_latex(index = False, column_format= "c", decimal=','))

Der Mittelwert für n beträgt: 4.776290067736398 ± 8.888084422378968
\begin{tabular}{c}
\toprule
 Column\_A \\
\midrule
 2,189004 \\
 2,218190 \\
 2,211720 \\
 2,302198 \\
 2,322735 \\
 2,377642 \\
 2,518670 \\
 2,399442 \\
 2,589478 \\
 2,593267 \\
 2,646371 \\
 2,833082 \\
 2,901770 \\
 2,656563 \\
 3,278201 \\
 3,143644 \\
 3,249584 \\
 8,171412 \\
46,355221 \\
34,822183 \\
23,250316 \\
11,664424 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1950/1971346999.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(prin.to_latex(index = False, column_format= "c", decimal=','))


In [ ]:
np.savetxt('tables/a.txt', md.values, header='f/Hz U/V', fmt='%.3f')
f, U = np.genfromtxt('tables/a.txt', unpack=True, skip_header=1)

def g(f, T):
    w = 2 * np.pi * f
    return 1 / (np.sqrt(1 + w**2 * T**2))

para, pcov = curve_fit(g, f, U)
pcov = np.sqrt(np.diag(pcov))
a = para
fa = pcov 

ValueError: too many values to unpack (expected 2)

Fehler berechnen:

In [ ]:
ua = ufloat(a, fa) * 1e6
T = ua
print('T = (%.3f +- %.3f )10^-6' % (noms(T), stds(T)))

Plot erstellen:

In [ ]:

plt.plot(f, Uc, 'xr', markersize=6 , label = 'Messwerte', alpha=0.5)        # alpha: opacity
xx = np.linspace(100, 150000, 10**4)
plt.plot(xx, g(xx, a), '-b', linewidth = 1, label = 'Ausgleichsfunktion')

plt.xlabel(r'$f \, / \, \mathrm{Hz}$')
plt.ylabel(r'$A(\omega) \, / \, U_0$')
plt.xscale('log')                       # axis scale 
plt.legend(loc="best")                  # legend position
plt.grid(True)                          # grid style
plt.xlim(200, 150000)                   # limitation of visible scale in plot